## 1. 导入依赖并设置随机种子


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

## 2. 创建 Embedding 层

假设我们有一个小型词表（5 个词），每个词用 3 维向量表示。


In [2]:
# 假设词表大小为 5，每个词向量维度为 3
num_embeddings = 5
embedding_dim = 3

# 创建一个 Embedding 层
emb_layer = nn.Embedding(num_embeddings, embedding_dim)
print(f"Embedding 层的权重形状: {emb_layer.weight.shape}")
print(f"Embedding 层的权重类型: {type(emb_layer.weight)}")
print(f"\nEmbedding 权重矩阵:\n{emb_layer.weight}")

Embedding 层的权重形状: torch.Size([5, 3])
Embedding 层的权重类型: <class 'torch.nn.parameter.Parameter'>

Embedding 权重矩阵:
Parameter containing:
tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863],
        [ 2.2082, -0.6380,  0.4617],
        [ 0.2674,  0.5349,  0.8094],
        [ 1.1103, -1.6898, -0.9890]], requires_grad=True)


## 3. 比较两种实现方式

我们将比较两种获取词向量的方法：

- **方法 A**: 使用 `nn.Embedding` 直接查表
- **方法 B**: 使用 One-Hot 编码 + 矩阵乘法


In [3]:
# 输入：获取索引为 1 和 3 的词向量
input_indices = torch.tensor([1, 3])

# --- 方法 A: 使用 nn.Embedding (查表) ---
output_emb = emb_layer(input_indices)
print("方法 A (Embedding 查表) 输出:")
print(output_emb)
print(f"grad_fn: {output_emb.grad_fn}")

方法 A (Embedding 查表) 输出:
tensor([[ 0.2303, -1.1229, -0.1863],
        [ 0.2674,  0.5349,  0.8094]], grad_fn=<EmbeddingBackward0>)
grad_fn: <EmbeddingBackward0 object at 0x111ecf0a0>


In [4]:
# --- 方法 B: 手动模拟 (One-Hot + 矩阵乘法) ---
one_hot = F.one_hot(input_indices, num_classes=num_embeddings).float()
print("One-Hot 编码:")
print(one_hot)

output_matmul = torch.matmul(one_hot, emb_layer.weight)
print("\n方法 B (One-Hot + MatMul) 输出:")
print(output_matmul)
print(f"grad_fn: {output_matmul.grad_fn}")

One-Hot 编码:
tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.]])

方法 B (One-Hot + MatMul) 输出:
tensor([[ 0.2303, -1.1229, -0.1863],
        [ 0.2674,  0.5349,  0.8094]], grad_fn=<MmBackward0>)
grad_fn: <MmBackward0 object at 0x1116aca30>


### 验证数学等价性


In [5]:
# 验证两者是否完全相等
are_equal = torch.allclose(output_emb, output_matmul)
print(f"两者结果是否数学等价? {are_equal}")
print(f"\n最大差异: {torch.max(torch.abs(output_emb - output_matmul)).item():.10f}")

两者结果是否数学等价? True

最大差异: 0.0000000000


## 4. 反向传播的差异

虽然前向传播结果相同，但是 `grad_fn` 不同：

- `EmbeddingBackward0`: Embedding 查表的反向传播
- `MmBackward0`: 矩阵乘法的反向传播

让我们验证梯度是否相同：


In [ ]:
# 准备两个相同的权重矩阵
weight_data = torch.randn(5, 3)
W_emb = torch.nn.Parameter(weight_data.clone())
W_matmul = torch.nn.Parameter(weight_data.clone())

# 输入 ID
indices = torch.tensor([1, 3])
grad_output = torch.randn(2, 3)  # 假设传回来的梯度

print("初始权重矩阵:")
print(W_emb)
print("\n从上游传来的梯度:")
print(grad_output)

初始权重矩阵:
Parameter containing:
tensor([[ 0.9580,  1.3221,  0.8172],
        [-0.7658, -0.7506,  1.3525],
        [ 0.6863, -0.3278,  0.7950],
        [ 0.2815,  0.0562,  0.5227],
        [-0.2384, -0.0499,  0.5263]], requires_grad=True)

从上游传来的梯度:
tensor([[-0.0085,  0.7291,  0.1331],
        [ 0.8640, -1.0157, -0.8887]])


In [7]:
# --- 路径 A: Embedding ---
out_a = F.embedding(indices, W_emb)
out_a.backward(grad_output)

print("Embedding 的梯度 (W_emb.grad):")
print(W_emb.grad)

Embedding 的梯度 (W_emb.grad):
tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.0085,  0.7291,  0.1331],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.8640, -1.0157, -0.8887],
        [ 0.0000,  0.0000,  0.0000]])


In [8]:
# --- 路径 B: MatMul ---
one_hot = F.one_hot(indices, num_classes=5).float()
out_b = torch.matmul(one_hot, W_matmul)
out_b.backward(grad_output)

print("MatMul 的梯度 (W_matmul.grad):")
print(W_matmul.grad)

MatMul 的梯度 (W_matmul.grad):
tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.0085,  0.7291,  0.1331],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.8640, -1.0157, -0.8887],
        [ 0.0000,  0.0000,  0.0000]])


In [9]:
# 验证梯度是否相等
print(f"梯度数值是否完全相等? {torch.allclose(W_emb.grad, W_matmul.grad)}")
print(f"\n梯度最大差异: {torch.max(torch.abs(W_emb.grad - W_matmul.grad)).item():.10f}")

梯度数值是否完全相等? True

梯度最大差异: 0.0000000000


## 5. 为什么需要 EmbeddingBackward？

既然结果一样，为什么 PyTorch 要专门设计 `EmbeddingBackward` 而不直接用矩阵乘法？

### 原因 1: 计算效率

- Embedding: 直接索引，时间复杂度 O(1)
- MatMul: 需要先构造 One-Hot 矩阵，再进行矩阵乘法，时间复杂度 O(vocab_size)

### 原因 2: 稀疏梯度（Sparse Gradients）

这是最重要的原因！让我们演示一下：


In [ ]:
# 对比稠密梯度和稀疏梯度
print("=== 稠密梯度 (Dense) vs 稀疏梯度 (Sparse) ===")
print("\nMatMul 产生的梯度 (稠密):")
print(f"  - 形状: {W_matmul.grad.shape}")
print(f"  - 非零元素数量: {torch.count_nonzero(W_matmul.grad)}")
print(f"  - 内存占用: {W_matmul.grad.element_size() * W_matmul.grad.nelement()} 字节")

print("\nEmbedding 产生的梯度 (也是稠密的，但可以配置为稀疏):")
print(f"  - 形状: {W_emb.grad.shape}")
print(f"  - 非零元素数量: {torch.count_nonzero(W_emb.grad)}")
print(f"  - 内存占用: {W_emb.grad.element_size() * W_emb.grad.nelement()} 字节")

=== 稠密梯度 (Dense) vs 稀疏梯度 (Sparse) ===

MatMul 产生的梯度 (稠密):
  - 形状: torch.Size([5, 3])
  - 非零元素数量: 6
  - 内存占用: 60 字节

Embedding 产生的梯度 (也是稠密的，但可以配置为稀疏):
  - 形状: torch.Size([5, 3])
  - 非零元素数量: 6
  - 内存占用: 60 字节


### 稀疏梯度的重要性

当使用 `nn.Embedding(sparse=True)` 时：

- **MatMul 的梯度**：稠密张量（Dense Tensor），即使只更新了 2 个词，也会生成一个和整个词表一样大的矩阵（比如 10 万行），其中 99,998 行都是 0。这非常占显存。
- **Embedding 的梯度**：稀疏张量（Sparse Tensor），只记录 `(index=1, value=...), (index=3, value=...)`，不存储那些 0。

**对于超大词表（如几百万词）的训练，稀疏梯度至关重要！**


## 6. 总结

| 特性           | Embedding 查表       | One-Hot + MatMul       |
| -------------- | -------------------- | ---------------------- |
| **前向计算**   | O(1) 直接索引        | O(vocab_size) 矩阵运算 |
| **反向传播**   | `EmbeddingBackward0` | `MmBackward0`          |
| **梯度类型**   | 可配置稀疏梯度       | 始终是稠密梯度         |
| **内存效率**   | 高（稀疏模式下）     | 低（大词表时）         |
| **数学等价性** | ✅ 完全等价          | ✅ 完全等价            |
| **适用场景**   | 大规模 NLP 任务      | 教学演示               |

### 关键要点

1. **数学上完全等价**：Embedding 查表 = One-Hot 编码 + 矩阵乘法
2. **实现上大不相同**：Embedding 专门优化了索引操作和稀疏梯度
3. **工程上的选择**：对于大词表，使用 `nn.Embedding(sparse=True)` 可以节省大量内存
